In [1]:
from bs4 import BeautifulSoup

In [2]:
DATA_PATH = "parsed.html"

In [3]:
soup = BeautifulSoup(open(DATA_PATH), "html.parser")
cls_2 = soup.find_all('div',class_="cls_002") # find all divs that contain a bold line

In [4]:
# All divs have text, and the attribute top in the style tells us if two divs were on the same line in the pdf

def get_top_value(style):
    # top value is always at the end of the style string, with a trailing px
    top = []
    for i in range(len(style) - 3, -1, -1): # ignore trailing px
        if style[i] == ":": break
        top.append(style[i])
    return "".join(top[::-1])

text_top = []
for div in cls_2:
    text = div.text
    top = get_top_value(div['style'])
    text_top.append((text, top))

In [5]:
# Mostly there are two data points per top value (code + description), but sometimes there are three when there is a symbol (* + code + description)
# We ignore the symbol, also we check sequentially for same top values to merge text since they repeat themselves over the pages

merged_text = []
for i in range(len(text_top) - 2):
    if text_top[i][1] != text_top[i + 1][1]: # if the next top is != than me, I need to check if there is any merge option behind with same top value or someone behind me, this be triggered for the symbol in triple cases
        txt = text_top[i][0]
        if i != 0 and text_top[i - 1][1] == text_top[i][1]: 
            txt += " " + text_top[i - 1][0]
        merged_text.append(txt)
# Add the last two data points that were not considered
merged_text.append(text_top[len(text_top) - 2][0])
merged_text.append(text_top[len(text_top) - 1][0])

In [7]:
# Remove special symbols
f = []
for x in merged_text:
    f.append(''.join(e for e in x if e.isalnum() or e == "." or e == " "))

In [9]:
# separate codes from descriptions
code_desc = []
for x in f:
    if not x:
        continue
    i = 0 
    while x[i] == " ": # remove leading spaces
        i += 1
    number = ""
    desc = ""
    if x and x[i].isdigit(): # if string starts with digit parse number first
        while i < len(x):
            if not (x[i].isdigit() or x[i] == "."):
                break 
            number += x[i]
            i += 1
        while i < len(x):
            desc += x[i]
            i += 1
    else:
        while i < len(x):
            if (x[i].isdigit() or x[i] == "."):
                break 
            desc += x[i]
            i += 1
        while i < len(x):
            number += x[i]
            i += 1 
    if number and desc: # sometimes there are empty lines
        if 1349 <= float(number) <= 1563: # remove page numbers
            continue
        # remove leading and trailing spaces
        desc = desc.strip().lstrip(' ')
        number = number.strip().lstrip(' ')
        code_desc.append([number, desc])

In [11]:
# write to csv file
with open('codes_desc.csv','w', encoding='Latin-1') as f:
    for n,d in code_desc:
        f.write("{},{}\n".format(n, d))

In [14]:
# example of how to open the file
# its important to check that trailing zeros are still there and numbers are not rounded (excel removes them for instance):
# 12.00 Extracción de cuerpo extraño intraocular del segmento anterior del ojo no especificada de
# 12.01 Extracción de cuerpo extraño intraocular del segmento anterior del ojo con empleo de imán
# 12.02 Extracción de cuerpo extraño del segmento anterior del ojo sin empleo de imán
# and not 12,12,12
with open('codes_desc.csv','r', encoding='Latin-1') as f:
    for l in f.readlines():
        n, d = l.split(",")
        if n[:2] == "12":
            print(n)

12
12.0
12.00
12.01
12.02
12.1
12.11
12.12
12.13
12.14
12.2
12.21
12.22
12.29
12.3
12.31
12.32
12.33
12.34
12.35
12.39
12.4
12.40
12.41
12.42
12.43
12.44
12.5
12.51
12.52
12.53
12.54
12.55
12.59
12.6
12.61
12.62
12.63
12.64
12.65
12.66
12.67
12.69
12.7
12.71
12.72
12.73
12.74
12.79
12.8
12.81
12.82
12.83
12.84
12.85
12.86
12.87
12.88
12.89
12.9
12.91
12.92
12.93
12.97
12.98
12.99
12.
